## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-14-14-45-39 +0000,nyt,"After Fiery Protests, Nepal’s Leader Promises ...",https://www.nytimes.com/2025/09/14/world/asia/...
1,2025-09-14-14-41-59 +0000,nyt,Nepal’s Social Media Ban Backfires as Politics...,https://www.nytimes.com/2025/09/11/world/asia/...
2,2025-09-14-14-40-03 +0000,nyt,"McCaul, a Foreign Policy Leader in the House, ...",https://www.nytimes.com/2025/09/14/us/politics...
3,2025-09-14-14-38-15 +0000,nypost,Spectacular drone show over Vatican recreates ...,https://nypost.com/2025/09/14/world-news/vatic...
4,2025-09-14-14-35-48 +0000,nyt,"Ricky Hatton, British Boxer and Former World C...",https://www.nytimes.com/2025/09/14/sports/rick...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2339/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
110,charlie,28
111,kirk,28
68,trump,20
309,new,9
96,russia,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
15,2025-09-14-13-57-20 +0000,nypost,Charlie Kirk suspect Tyler Robinson not cooper...,https://nypost.com/2025/09/14/us-news/charlie-...,95
139,2025-09-13-20-58-19 +0000,nypost,"Jasmine Crockett denies ‘wannabe Hitler,’ left...",https://nypost.com/2025/09/13/us-news/jasmine-...,93
92,2025-09-14-03-44-53 +0000,nypost,Charlie Kirk’s alleged assassin Tyler Robinson...,https://nypost.com/2025/09/13/us-news/charlie-...,91
131,2025-09-13-22-23-48 +0000,nypost,Charlie Kirk’s accused assassin Tyler Robinson...,https://nypost.com/2025/09/13/us-news/accused-...,87
87,2025-09-14-04-20-55 +0000,nypost,Oxford Union president-elect will be disciplin...,https://nypost.com/2025/09/14/world-news/oxfor...,87


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
15,95,2025-09-14-13-57-20 +0000,nypost,Charlie Kirk suspect Tyler Robinson not cooper...,https://nypost.com/2025/09/14/us-news/charlie-...
94,71,2025-09-14-03-00-00 +0000,wsj,Israeli Prime Minister Benjamin Netanyahu and ...,https://www.wsj.com/politics/national-security...
16,37,2025-09-14-13-48-20 +0000,cbc,Ukrainian drones strike one of Russia's top oi...,https://www.cbc.ca/news/world/ukraine-drones-r...
22,33,2025-09-14-13-00-00 +0000,wsj,"As America’s 250th birthday approaches, Robert...",https://www.wsj.com/business/robert-reed-churc...
57,32,2025-09-14-09-30-00 +0000,wsj,In the weeks since President Trump’s 50% tarif...,https://www.wsj.com/economy/trade/global-trade...
97,30,2025-09-14-02-56-48 +0000,nypost,Warner Bros. Discovery shares spike as CEO Dav...,https://nypost.com/2025/09/13/business/warner-...
135,30,2025-09-13-21-32-26 +0000,nypost,Israel bombs Hamas operatives in another Gaza ...,https://nypost.com/2025/09/13/world-news/israe...
27,24,2025-09-14-12-40-42 +0000,nypost,NJ judge rejects bid to free Colt’s Neck quadr...,https://nypost.com/2025/09/14/us-news/colts-ne...
87,24,2025-09-14-04-20-55 +0000,nypost,Oxford Union president-elect will be disciplin...,https://nypost.com/2025/09/14/world-news/oxfor...
105,23,2025-09-14-01-00-00 +0000,wsj,The unemployment rate is low. But a measure th...,https://www.wsj.com/economy/jobs/underemployed...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
